# 🌌 x.titan: Autonomous Trading Brain - Kaggle Training

**Version**: 2026.1.2 (GPU-Optimized)
**Source**: GitHub → https://github.com/planetazul3/x.titan

## ⚙️ Requirements
1. **GPU Accelerator**: Enable in Kaggle Settings (T4 or P100 recommended)
2. **Kaggle Secrets**: Add `DERIV_API_TOKEN` for data download
3. **Internet**: Enable for GitHub clone + pip install

## 1. Clone Repository from GitHub

In [ ]:
import os
from pathlib import Path

REPO_URL = "https://github.com/planetazul3/x.titan.git"
WORKING_DIR = Path("/kaggle/working/x.titan")

if not WORKING_DIR.exists():
    print("📥 Cloning x.titan from GitHub...")
    !git clone --depth 1 {REPO_URL} {WORKING_DIR}
else:
    print("✅ Repository already exists")

os.chdir(WORKING_DIR)
print(f"📂 Working directory: {os.getcwd()}")

## 2. TA-Lib Installation (C Library)

In [ ]:
if not os.path.exists('/usr/include/ta-lib/ta_defs.h'):
    print("📦 Installing TA-Lib...")
    !apt-get update -qq && apt-get install -y -qq build-essential wget
    !wget -q http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
    !tar -xzf ta-lib-0.4.0-src.tar.gz
    os.chdir('ta-lib')
    !./configure --prefix=/usr > /dev/null && make -j4 > /dev/null && make install > /dev/null
    os.chdir('..')
    !rm -rf ta-lib ta-lib-0.4.0-src.tar.gz
    print("✅ TA-Lib compiled")
else:
    print("✅ TA-Lib already installed")

## 3. Python Dependencies

In [ ]:
print("📥 Installing Python packages...")
!pip install -q TA-Lib pandas numpy torch tqdm pydantic pydantic-settings python-dotenv pyarrow
!pip install -q -e ./python-deriv-api 2>/dev/null || pip install -q git+https://github.com/planetazul3/python-deriv-api.git
print("✅ Dependencies installed")

## 4. Environment Configuration (API Key via Kaggle Secrets)

**⚠️ IMPORTANT**: Add your Deriv API token in Kaggle:
1. Click **Add-ons** → **Secrets**
2. Add key: `DERIV_API_TOKEN` with your token value

In [ ]:
from kaggle_secrets import UserSecretsClient

# Load API token from Kaggle secrets
try:
    secrets = UserSecretsClient()
    DERIV_API_TOKEN = secrets.get_secret("DERIV_API_TOKEN")
    print("✅ DERIV_API_TOKEN loaded from Kaggle secrets")
except Exception as e:
    print(f"⚠️ Could not load secret: {e}")
    DERIV_API_TOKEN = ""  # Will fail on data download

# Create .env file with all configuration
env_content = f'''# x.titan Configuration (Generated by Kaggle Notebook)
ENVIRONMENT=test
DERIV_API_TOKEN={DERIV_API_TOKEN}
DERIV_APP_ID=1089

# Trading
TRADING__SYMBOL=R_100
TRADING__STAKE_AMOUNT=10.0

# Probability Thresholds
THRESHOLDS__CONFIDENCE_THRESHOLD_HIGH=0.80
THRESHOLDS__LEARNING_THRESHOLD_MAX=0.70
THRESHOLDS__LEARNING_THRESHOLD_MIN=0.50

# TFT Hyperparameters (GPU Optimized)
HYPERPARAMS__USE_TFT=True
HYPERPARAMS__LEARNING_RATE=0.0007
HYPERPARAMS__BATCH_SIZE=128
HYPERPARAMS__LSTM_HIDDEN_SIZE=256
HYPERPARAMS__CNN_FILTERS=128
HYPERPARAMS__LATENT_DIM=64
HYPERPARAMS__DROPOUT_RATE=0.2
HYPERPARAMS__EPOCHS=30

# Data Shapes
DATA_SHAPES__SEQUENCE_LENGTH_TICKS=1000
DATA_SHAPES__SEQUENCE_LENGTH_CANDLES=200
DATA_SHAPES__WARMUP_STEPS=50
'''

with open('.env', 'w') as f:
    f.write(env_content)
print("✅ .env file created")

## 5. GPU Detection & Optimization

In [ ]:
import torch

print("=" * 50)
print("GPU DETECTION")
print("=" * 50)

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_mem = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"✅ GPU: {gpu_name}")
    print(f"   VRAM: {gpu_mem:.1f} GB")
    
    # Optimize for available VRAM
    if gpu_mem >= 15:  # T4/P100 with 16GB
        os.environ['HYPERPARAMS__BATCH_SIZE'] = '256'
        print("   Batch size: 256 (high VRAM)")
    else:
        os.environ['HYPERPARAMS__BATCH_SIZE'] = '64'
        print("   Batch size: 64 (limited VRAM)")
    
    !nvidia-smi
else:
    print("⚠️ NO GPU DETECTED - Training will be very slow!")
    os.environ['HYPERPARAMS__BATCH_SIZE'] = '16'

## 6. Data Download (12 Months)

In [ ]:
DATA_PATH = Path('./data_cache')

# Check if data already exists
existing_files = list(DATA_PATH.glob('**/ticks/*.parquet')) if DATA_PATH.exists() else []

if len(existing_files) >= 6:
    print(f"✅ Found {len(existing_files)} tick files - skipping download")
else:
    print("📡 Downloading 12 months of historical data...")
    print("   (This takes 10-15 minutes)")
    !python scripts/download_data.py --months 12 --symbol R_100 --output data_cache

## 7. Training Pipeline

In [ ]:
import time

# Clean previous runs
!rm -rf checkpoints/*.pt 2>/dev/null
!rm -rf logs/tensorboard/* 2>/dev/null

print("🚀 STARTING TRAINING")
print("=" * 50)
start_time = time.time()

# Execute training with GPU optimizations
# - pin_memory=True for faster CPU→GPU transfer
# - num_workers auto-adjusted based on CPU count
!python scripts/train.py \
    --data-path data_cache \
    --epochs 30 \
    --checkpoint-dir checkpoints

elapsed = time.time() - start_time
print(f"\n⏱️ Training completed in {elapsed/60:.1f} minutes")

## 8. Verify & Export Model

In [ ]:
import shutil
from datetime import datetime

# Verify checkpoint
best_model = Path('checkpoints/best_model.pt')
if best_model.exists():
    size_mb = best_model.stat().st_size / 1e6
    print(f"✅ best_model.pt: {size_mb:.1f} MB")
    
    # Validate checkpoint
    !python tools/verify_checkpoint.py --checkpoint checkpoints/best_model.pt
else:
    print("❌ No checkpoint found!")

# Create downloadable bundle
ts = datetime.now().strftime('%Y%m%d_%H%M')
bundle = f'xtitan_model_{ts}'
shutil.make_archive(bundle, 'zip', root_dir='.', base_dir='checkpoints')
print(f"\n📦 Download: {bundle}.zip from Output panel")